In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:

# Define constants
BASE_DIR = './dataset/train'

# * Number of times we are going to run through the entire dataset.
EPOCHS = 10  
# * the image size that we are going to set the images in the dataset to.
IMAGE_SIZE = 224
# * the number of images we are inputting into the neural network at once.
BATCH_SIZE = 64
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

RESULT_LABELS = 'results/labels.txt'


In [ ]:
# Set up data generator with data augmentation techniques such as rotation, shifting and flipping
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # rescale pixel values to [0,1]
    validation_split=0.2,  # reserve some data for validation
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
)

In [ ]:
# Generate training data
train_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'  # specify this is training data
)

In [ ]:
# Generate validation data
val_generator = datagen.flow_from_directory(
    BASE_DIR,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'  # specify this is validation data
)

In [ ]:
# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # monitor the validation loss
    patience=5,  # number of epochs with no improvement after which training will be stopped
    verbose=1,  # report the early stopping events
    restore_best_weights=True  # restore the best weights from the monitored quantity
)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # monitor the validation loss
    factor=0.2,  # factor by which the learning rate will be reduced
    patience=3,  # number of epochs with no improvement after which learning rate will be reduced
    verbose=1,  # report the lr reduction events
    min_lr=0.0001  # lower bound on the learning rate
)

In [ ]:
# Print and save the class indices
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open(RESULT_LABELS, 'w') as f:
    f.write(labels)

In [ ]:
# Define the device strategy
strategy = tf.distribute.OneDeviceStrategy("GPU:0")

# Define the model within the strategy scope
with strategy.scope():
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=IMAGE_SHAPE,
        include_top=False,  # exclude the top layer
    )
    base_model.trainable = False  # freeze the base model

    # Define the custom top layers
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Conv2D(64, 3, activation='relu'),  # increased filter size for more complex patterns
        tf.keras.layers.BatchNormalization(),  # normalize the activations of the previous layer at each batch
        tf.keras.layers.Dropout(0.2),  # randomly set a fraction rate of input units to 0 at each update during training
        tf.keras.layers.Conv2D(64, 3, activation='relu'),  # another convolutional layer
        tf.keras.layers.BatchNormalization(),  # batch normalization layer
        tf.keras.layers.GlobalAveragePooling2D(),  # apply average pooling on the spatial dimensions until each spatial dimension is one
        tf.keras.layers.Dense(128, activation='relu'),  # dense layer with more neurons
        tf.keras.layers.Dense(36, activation='softmax')  # final softmax layer
    ])

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),  # use Adam optimizer with initial learning rate 0.001
        loss='categorical_crossentropy',  # use categorical cross entropy as the loss function
        metrics=['accuracy']  # use accuracy as the metric
    )

### Train the model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]  # add the callbacks to the training process
)

### Save the trained model

In [ ]:
# Save the model
saved_model_dir = 'results'
tf.saved_model.save(model, saved_model_dir)

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save the TFLite model
with open('results/model.tflite', 'wb') as f:
    f.write(tflite_model)